# PlanLLM usage

In [11]:
import json
import recipe_parser as rp
import os
import requests
import random

In [12]:
recipe_len = len(rp.get_recipes())


recipe_num = str(random.randint(0, recipe_len - 1))

print (f"Number of recipes: {recipe_len}")
print (f"Recipe number {recipe_num}")

displayName = rp.get_recipes()[recipe_num]["displayName"]
instructions = rp.get_recipes()[recipe_num]["instructions"]
numInstructions = len(instructions)

instructionsText = []

for i in range(numInstructions):
    instructionsText.append(instructions[i]["stepText"])


with open('conversation.json') as f:
    conversation = json.load(f)

conversation['task']['recipe']['displayName'] = displayName
conversation['task']['recipe']['instructions'] = [{'stepText': instruction} for instruction in instructionsText]
conversation['dialog'] = []

with open('conversation.json', 'w') as f:
    json.dump(conversation, f)

conversation


Number of recipes: 994
Recipe number 187


{'dialog_id': '1',
 'system_tone': 'neutral',
 'task': {'recipe': {'displayName': 'How to Cut and Core Cauliflower',
   'instructions': [{'stepText': 'Pull leaves off base of cauliflower.'},
    {'stepText': 'Cut cauliflower in half from the top of the crown through the stem.'},
    {'stepText': 'Lay the cauliflower halves cut side down on the cutting board and cut them vertically again.'},
    {'stepText': 'Lay the cauliflower quarter on the cutting board and angle the knife so that it cuts out the core. Repeat with remaining quarters of cauliflower.'},
    {'stepText': 'Pull apart the cauliflower florets.'}]}},
 'dialog': []}

In [13]:
external_url = "https://twiz.novasearch.org/"
max_timeout = 10

def test_structured_post_request(url: str, conversation):

    url = os.path.join(url, "structured")

    # check this file to understand the structure of the data
    # with open('conversation.json') as f:
    #     data = json.load(f)

    data = {
        "dialog": conversation,
        "max_tokens": 100,
        "temperature": 0.0,
        "top_p": 1,
        "top_k": -1,
    }

    # Make the POST request
    response = requests.post(url, json=data, timeout=max_timeout)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        return response.text
    else:
        print("POST request failed with status code:", response.status_code)

current_step = 0
user = input("Ask something!")
while (user != "quit()"):
    print(user)

    with open('conversation.json') as f:
        conversation = json.load(f)

    conversation['dialog'].append({'current_step' : current_step, 'user' : user})

    response = test_structured_post_request(external_url, conversation)

    print(response)

    conversation['dialog'][current_step]['system'] = response

    with open('conversation.json', 'w') as f:
        json.dump(conversation, f)

    user = input()
    current_step += 1

Whats the recipe about
"Let's get cooking! Step 1: Pull leaves off base of cauliflower. "

Give me the ingredients
"The ingredients for this recipe are: \u2753 Cauliflower \u2753 Leaves \u2753 Knife \u2753 Board \u2753 Cutting board \u2753 Kitchen Shears "

give me the first step
"Step 2: Cut cauliflower in half from the top of the crown through the stem. "

How do I start the recipe
"Ready to dive in? Step 1: Pull leaves off base of cauliflower. "

nexty
"Step 2: Cut cauliflower in half from the top of the crown through the stem. "

next
"Step 3: Lay the cauliflower halves cut side down on the cutting board and cut them vertically again. "

exit()
"Sure, I will take you back to the search interface where you can continue your search or look for another task. Is there anything else I can assist you with? "

exit()
"Sure, I'll take you back to the list of recipes or DIY tasks. Is there anything else you need help with or would you like to continue with a different task? "

